## 정답 label 전처리

- 336시간의 예측값을 도출해내기 위해서는 336시간의 정답 label이 필요
    - 주어진 train 데이터를 기반으로 행별 336시간의 적산차 값 데이터 셋 생성

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/bjhkh/Downloads/data_city/data_tr_city.csv')
df

,datetime,구미 혁신도시배수지 유출유량 적산차
0,2017-01-01 01:00:00,138.0
1,2017-01-01 02:00:00,237.0
2,2017-01-01 03:00:00,128.0
3,2017-01-01 04:00:00,14.0
4,2017-01-01 05:00:00,11.0
...,...,...
35058,2020-12-31 19:00:00,328.0
35059,2020-12-31 20:00:00,347.0
35060,2020-12-31 21:00:00,335.0
35061,2020-12-31 22:00:00,141.0


In [3]:
df[df['datetime'].duplicated() == True]

,datetime,구미 혁신도시배수지 유출유량 적산차


In [4]:
df.isnull().sum()

datetime               0
구미 혁신도시배수지 유출유량 적산차    8
dtype: int64

# 결측치 처리

In [5]:
df_x = pd.DataFrame(df['구미 혁신도시배수지 유출유량 적산차'])

In [6]:
#결측치 처리
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter = 10, random_state = 0)
df1 = pd.DataFrame(imputer.fit_transform(df_x))

df1.columns = df_x.columns
df1

,구미 혁신도시배수지 유출유량 적산차
0,138.0
1,237.0
2,128.0
3,14.0
4,11.0
...,...
35058,328.0
35059,347.0
35060,335.0
35061,141.0


In [7]:
df = pd.concat([df['datetime'], df1], axis=1)
df

,datetime,구미 혁신도시배수지 유출유량 적산차
0,2017-01-01 01:00:00,138.0
1,2017-01-01 02:00:00,237.0
2,2017-01-01 03:00:00,128.0
3,2017-01-01 04:00:00,14.0
4,2017-01-01 05:00:00,11.0
...,...,...
35058,2020-12-31 19:00:00,328.0
35059,2020-12-31 20:00:00,347.0
35060,2020-12-31 21:00:00,335.0
35061,2020-12-31 22:00:00,141.0


In [8]:
df.isnull().sum()

datetime               0
구미 혁신도시배수지 유출유량 적산차    0
dtype: int64

# 이상값 직전값으로 대체

In [9]:
q1 = df['구미 혁신도시배수지 유출유량 적산차'].quantile(0.25)
q3 = df['구미 혁신도시배수지 유출유량 적산차'].quantile(0.75)
iqr = q3-q1
low = q1-1.75*iqr
high = q3+1.75*iqr

In [10]:
q3

346.0

In [11]:
outlier = df[(df['구미 혁신도시배수지 유출유량 적산차']<low) | (df['구미 혁신도시배수지 유출유량 적산차']>high)].index

In [12]:
## 직전 값으로 대체
from tqdm import tqdm
for i in outlier:
    df['구미 혁신도시배수지 유출유량 적산차'][i] = df['구미 혁신도시배수지 유출유량 적산차'][i-1]

## 잘 대체되었는지 확인
print(df.loc[outlier, :])

C:\Users\bjhkh\AppData\Local\Temp\ipykernel_2136\2473219928.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['구미 혁신도시배수지 유출유량 적산차'][i] = df['구미 혁신도시배수지 유출유량 적산차'][i-1]


                  datetime  구미 혁신도시배수지 유출유량 적산차
2127   2017-03-30 16:00:00                172.0
2866   2017-04-30 11:00:00                  0.0
3468   2017-05-25 13:00:00                  0.0
3485   2017-05-26 06:00:00                  0.0
3491   2017-05-26 12:00:00                  0.0
...                    ...                  ...
32243  2020-09-05 12:00:00                249.0
32707  2020-09-24 20:00:00                299.0
33309  2020-10-19 22:00:00                256.0
34316  2020-11-30 21:00:00                380.0
34484  2020-12-07 21:00:00                474.0

[63 rows x 2 columns]


# train_정답label 데이터 셋 생성

In [13]:
df.tail(336)

,datetime,구미 혁신도시배수지 유출유량 적산차
34727,2020-12-18 00:00:00,191.0
34728,2020-12-18 01:00:00,180.0
34729,2020-12-18 02:00:00,147.0
34730,2020-12-18 03:00:00,65.0
34731,2020-12-18 04:00:00,56.0
...,...,...
35058,2020-12-31 19:00:00,328.0
35059,2020-12-31 20:00:00,347.0
35060,2020-12-31 21:00:00,335.0
35061,2020-12-31 22:00:00,141.0


In [14]:
from tqdm import tqdm
b = []
i = 0
for i in tqdm(range(34727)) :
    a = df[['datetime', '구미 혁신도시배수지 유출유량 적산차']].iloc[i:337+i].transpose().iloc[1:].values
    b.append(a)

100%|██████████| 34727/34727 [02:36<00:00, 221.68it/s]


In [15]:
len(b)

34727

In [16]:
b[34726]

array([[392.0, 191.0, 180.0, 147.0, 65.0, 56.0, 53.0, 35.0, 114.0, 178.0,
        442.0, 308.0, 346.0, 538.0, 474.0, 470.0, 331.0, 310.0, 293.0,
        335.0, 451.0, 404.0, 433.0, 366.0, 109.0, 190.0, 161.0, 141.0,
        122.0, 73.0, 43.0, 47.0, 51.0, 55.0, 176.0, 320.0, 433.0, 554.0,
        409.0, 121.0, 324.0, 375.0, 274.0, 216.0, 278.0, 300.0, 362.0,
        214.0, 209.0, 174.0, 132.0, 140.0, 55.0, 46.0, 34.0, 26.0, 77.0,
        231.0, 121.0, 189.0, 215.0, 428.0, 523.0, 514.0, 740.0, 434.0,
        196.0, 148.0, 485.0, 440.0, 383.0, 395.0, 201.0, 143.0, 188.0,
        97.0, 74.0, 58.0, 46.0, 32.0, 27.0, 308.0, 444.0, 423.0, 301.0,
        394.0, 481.0, 292.0, 251.0, 149.0, 178.0, 280.0, 307.0, 334.0,
        496.0, 302.0, 95.0, 278.0, 140.0, 98.0, 54.0, 44.0, 43.0, 77.0,
        69.0, 352.0, 394.0, 408.0, 517.0, 337.0, 297.0, 441.0, 606.0,
        226.0, 172.0, 240.0, 168.0, 365.0, 487.0, 567.0, 227.0, 162.0,
        171.0, 180.0, 56.0, 47.0, 99.0, 74.0, 87.0, 425.0, 423.0, 371

In [17]:
import numpy as np
b = np.array(b)
b
m,n,r = b.shape
out_arr = np.column_stack((np.repeat(np.arange(m),n),b.reshape(m*n,-1)))
b = pd.DataFrame(out_arr)
b

,0,1,2,3,4,5,6,7,8,9,...,328,329,330,331,332,333,334,335,336,337
0,0,138.0,237.0,128.0,14.0,11.0,10.0,11.0,187.0,392.0,...,450.0,315.0,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0
1,1,237.0,128.0,14.0,11.0,10.0,11.0,187.0,392.0,114.0,...,315.0,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0
2,2,128.0,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,...,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0
3,3,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,...,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0,26.0
4,4,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,250.0,...,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0,26.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34722,34722,322.0,286.0,377.0,426.0,392.0,191.0,180.0,147.0,65.0,...,387.0,368.0,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0
34723,34723,286.0,377.0,426.0,392.0,191.0,180.0,147.0,65.0,56.0,...,368.0,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0
34724,34724,377.0,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,...,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0,335.0
34725,34725,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,35.0,...,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0,335.0,141.0


In [18]:
b.columns

RangeIndex(start=0, stop=338, step=1)

In [19]:
b.drop(columns=[0,1], inplace=True)

In [20]:
name = []
for i in range(336) :
    name.append('h%d'%i)
    
name[:10]

['h0', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9']

In [21]:
b.columns = name

In [22]:
b

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h326,h327,h328,h329,h330,h331,h332,h333,h334,h335
0,237.0,128.0,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,...,450.0,315.0,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0
1,128.0,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,...,315.0,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0
2,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,250.0,...,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0
3,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,250.0,181.0,...,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0,26.0
4,10.0,11.0,187.0,392.0,114.0,243.0,203.0,250.0,181.0,141.0,...,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0,26.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34722,286.0,377.0,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,...,387.0,368.0,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0
34723,377.0,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,35.0,...,368.0,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0
34724,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,35.0,114.0,...,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0,335.0
34725,392.0,191.0,180.0,147.0,65.0,56.0,53.0,35.0,114.0,178.0,...,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0,335.0,141.0


In [23]:
df = pd.concat([df[['datetime', '구미 혁신도시배수지 유출유량 적산차']],b], axis=1)
df = df.head(34727)
df


,datetime,구미 혁신도시배수지 유출유량 적산차,h0,h1,h2,h3,h4,h5,h6,h7,...,h326,h327,h328,h329,h330,h331,h332,h333,h334,h335
0,2017-01-01 01:00:00,138.0,237.0,128.0,14.0,11.0,10.0,11.0,187.0,392.0,...,450.0,315.0,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0
1,2017-01-01 02:00:00,237.0,128.0,14.0,11.0,10.0,11.0,187.0,392.0,114.0,...,315.0,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0
2,2017-01-01 03:00:00,128.0,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,...,276.0,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0
3,2017-01-01 04:00:00,14.0,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,...,221.0,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0,26.0
4,2017-01-01 05:00:00,11.0,10.0,11.0,187.0,392.0,114.0,243.0,203.0,250.0,...,56.0,203.0,257.0,164.0,114.0,35.0,32.0,29.0,26.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34722,2020-12-17 19:00:00,322.0,286.0,377.0,426.0,392.0,191.0,180.0,147.0,65.0,...,387.0,368.0,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0
34723,2020-12-17 20:00:00,286.0,377.0,426.0,392.0,191.0,180.0,147.0,65.0,56.0,...,368.0,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0
34724,2020-12-17 21:00:00,377.0,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,...,173.0,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0,335.0
34725,2020-12-17 22:00:00,426.0,392.0,191.0,180.0,147.0,65.0,56.0,53.0,35.0,...,347.0,335.0,204.0,254.0,473.0,305.0,328.0,347.0,335.0,141.0


In [24]:
df[df['datetime'].duplicated() == True]

,datetime,구미 혁신도시배수지 유출유량 적산차,h0,h1,h2,h3,h4,h5,h6,h7,...,h326,h327,h328,h329,h330,h331,h332,h333,h334,h335


In [241]:
df.to_csv('C:/Users/bjhkh/OneDrive/바탕 화면/정답_label_v2.csv', index=False)